In [51]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as itr
import time
from boundaries import WallBoundary

# Initializing a rank three tensor filled with nodes
latticeSize = 9
xResolution = 5
yResolution = 5
relaxationTime = 0.5
# Weights
unitVect = np.array([
	[0, 0],	[1, 0], [0, 1],
	[-1, 0], [0, -1], [1, 1],
	[-1, 1], [-1, -1], [1, -1]
])
unitX = np.array([
	0, 1, 0, -1, 0, 1, -1, -1, 1
])
unitY = np.array([
	0, 0, 1, 0, -1, 1, 1, -1, -1
])
weight = np.array([
    4/9, 1/9, 1/9,
    1/9, 1/9, 1/36,
    1/36, 1/36, 1/36
])
cs = np.sqrt(3)

# Initializing the fluid matrix
fluid = np.ones((xResolution, yResolution, latticeSize)) + 0.1 * np.random.randn(xResolution, yResolution, latticeSize)
# Assigning a right velocity
fluid[:, :, 1] = 2.3

# Set a boundary
boundary = WallBoundary(xResolution, yResolution, False)
boundary.cylindricalWall([xResolution/2, yResolution/2], 2)

In [52]:
# One iteration only
iteratedFluid = np.ones((xResolution, yResolution, latticeSize))

# Internal collision step
density = np.sum(fluid, axis = 2)
mesoMomentum = fluid.reshape(xResolution, yResolution, latticeSize, 1) * unitVect
momentum = np.sum(mesoMomentum, axis = 2) / density.reshape(xResolution, yResolution, 1)
momentumMagnitude = np.linalg.norm(momentum, axis = 2) ** 2 / (2 * cs**4) # Represents u . u

# I kinda give up on broadcasting rn. Help me :skull:
for xIndex, yIndex in itr.product(range(xResolution), range(yResolution)):
    latticeDensity = density[xIndex, yIndex]
    cellMomentumMagnitude = momentumMagnitude[xIndex, yIndex]
    for latticeIndex in range(latticeSize):
        momentumDot = np.dot(unitVect[latticeIndex], momentum[xIndex, yIndex])
        fluidEquilibrium = latticeDensity * weight[latticeIndex] * (
			1 + momentumDot / (cs ** 2) + momentumDot ** 2 / (cs ** 4) - cellMomentumMagnitude
		)
        iteratedFluid[xIndex, yIndex, latticeIndex] = fluid[xIndex, yIndex, latticeIndex] * (1 - 1/relaxationTime) + 1/relaxationTime * fluidEquilibrium

iteratedFluid

array([[[ 8.53412490e+00,  1.45894996e-01,  1.28946779e+00,
          1.21661741e+00,  1.24738227e+00, -4.33838471e-01,
         -5.43986649e-01, -4.53202809e-01, -3.93868812e-01],
        [ 7.95861472e+00,  6.92598850e-02,  1.24768199e+00,
          1.20581983e+00,  1.32889673e+00, -3.87397068e-01,
         -4.46709601e-01, -3.77131960e-01, -4.50339856e-01],
        [ 8.39421271e+00,  1.04616854e-01,  1.19998352e+00,
          1.18324269e+00,  1.23839956e+00, -4.25142718e-01,
         -4.52216715e-01, -5.07394724e-01, -2.76383825e-01],
        [ 8.18370240e+00,  1.10482897e-01,  1.21720477e+00,
          1.23631351e+00,  1.36758435e+00, -3.12279161e-01,
         -3.27111089e-01, -5.77642957e-01, -5.17398467e-01],
        [ 8.06455103e+00,  4.11981412e-02,  1.09207476e+00,
          1.11577560e+00,  1.31340478e+00, -4.02432188e-01,
         -3.99750563e-01, -3.81263075e-01, -3.92508646e-01]],

       [[ 7.92757649e+00,  3.73961238e-03,  1.19059105e+00,
          1.15518152e+00,  1.1915

In [53]:
boundary.boundary

array([[False, False, False, False, False],
       [False, False,  True,  True, False],
       [False,  True,  True,  True,  True],
       [False,  True,  True,  True,  True],
       [False, False,  True,  True, False]])

In [58]:
density

array([[10.61332146, 10.15671952, 10.46335893, 10.38791277, 10.05864132],
       [ 9.8889203 , 10.41638957,  9.88705098, 10.02536562,  9.97280524],
       [10.21844173, 10.34429503, 10.54216577, 10.46944491, 10.39994713],
       [10.03914149, 10.16947397, 10.18106382, 10.27734808, 10.36451122],
       [ 9.99925623, 10.17844976, 10.82289509, 10.27458383, 10.40695544]])

In [54]:
def dontdoanything():
    x = 1

In [55]:
for latticeIndex, shiftX, shiftY in zip(range(latticeSize), unitX, unitY):
    iteratedFluid[:, :, latticeIndex] = np.roll(iteratedFluid[:, :, latticeIndex], shift = shiftX, axis = 1)
    iteratedFluid[:, :, latticeIndex] = np.roll(iteratedFluid[:, :, latticeIndex], shift = shiftY, axis = 0)

fluidBoundary = boundary.boundary

for x, biglist in enumerate(fluidBoundary):
    for y, isAWall in enumerate(biglist):
        if (isAWall):
            for latticeIndex in range(latticeSize):
                #for each vector in the lattice, send it back
                if (iteratedFluid[x, y, latticeIndex] != 0):
                    if latticeIndex in [3, 4, 7, 8]:
                        reflectIndex = latticeIndex - 2
                    else: 
                        reflectIndex = latticeIndex + 2
                    try: 
                        iteratedFluid[x - unitX[latticeIndex], y - unitY[latticeIndex], reflectIndex] += iteratedFluid[x, y, latticeIndex]
                    except:
                        dontdoanything()
                        
                    iteratedFluid[x, y, latticeIndex] = 0
                    

iteratedFluid

array([[[ 8.53412490e+00,  4.11981412e-02,  1.31885138e+00,
          1.20581983e+00,  1.19156488e+00, -4.95052675e-01,
         -4.20341008e-01, -6.32744918e-01, -2.32937382e-01],
        [ 7.95861472e+00,  1.45894996e-01,  1.14375825e+00,
          1.18324269e+00,  1.34453362e+00, -3.74237332e-01,
         -5.08744777e-01, -9.36209178e-01, -4.60211405e-01],
        [ 8.39421271e+00,  6.92598850e-02,  1.35064298e+00,
          1.33074949e+00,  1.32351451e+00, -3.56819093e-01,
         -3.70082881e-01, -8.69187691e-01, -3.45550744e-01],
        [ 8.18370240e+00,  1.04616854e-01,  1.15423238e+00,
          1.12455849e+00,  1.25149265e+00, -4.52241556e-01,
         -7.93276374e-01, -7.46640605e-01, -4.43071389e-01],
        [ 8.06455103e+00,  1.10482897e-01,  1.28914986e+00,
          1.21661741e+00,  1.28251652e+00, -4.74777820e-01,
         -8.62950102e-01, -4.79285624e-01, -4.02206330e-01]],

       [[ 7.92757649e+00, -1.53284175e-02,  1.28946779e+00,
          1.22605209e+00,  1.2648